In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Boolean, DateTime, ForeignKey, Float, Time, Table
import psycopg2
from sqlalchemy.orm import sessionmaker, declarative_base, relationship
from psycopg2 import extensions
from datetime import datetime
import os
from moviepy.editor import VideoFileClip
import pandas as pd


In [2]:
Base = declarative_base()

db_params = {
    'user': 'postgres',
    'password': 'Digilarger',  # enter your DB password
    'host': 'localhost',  # 'localhost' or IP address
    'port': '5432',  # '5432'
    'database': 'test', #tensionTerminator
}

print(f"This notebook will create a new Database with the name: {db_params['database']}\n")

connection = psycopg2.connect(
    user=db_params['user'],
    password=db_params['password'],
    host=db_params['host'],
    port=db_params['port'],
)
print(f"{connection}\n")

auto_comit = extensions.ISOLATION_LEVEL_AUTOCOMMIT
connection.set_isolation_level(auto_comit)
cursor = connection.cursor()
try:
    cursor.execute(f"CREATE DATABASE {db_params['database']}")
    connection.commit()
    cursor.close()
    connection.close()

except Exception as e:
    print(f"{e}")

db_url = f"postgresql://" \
            f"{db_params['user']}:" \
            f"{db_params['password']}@" \
            f"{db_params['host']}:" \
            f"{db_params['port']}/" \
            f"{db_params['database']}"

engine = create_engine(db_url, echo=True)
Session = sessionmaker(bind=engine)
session = Session()
print(f"Connected to PostgreSQL, DB: {db_params['database']}")

class Users(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    surename = Column(String)

    loops = relationship('Loop', back_populates='user')

#Relation Tables for M:M
tools_loop_association = Table(
    'tools_loop_association',
    Base.metadata,
    Column('tool_id', Integer, ForeignKey('tools.id')),
    Column('loop_id', Integer, ForeignKey('loop.id'))
)

bodyside_loop_association = Table(
    'bodyside_loop_association',
    Base.metadata,
    Column('bodyside_id', Integer, ForeignKey('bodyside.id')),
    Column('loop_id', Integer, ForeignKey('loop.id'))
)

class Tools(Base):
    __tablename__ = 'tools'

    id = Column(Integer, primary_key=True)
    tool = Column(String)
    time_in_use = Column(Time)

    #M:M
    loop = relationship('Loop', secondary=tools_loop_association, back_populates='tools')

class Bodyside(Base):
    __tablename__ = 'bodyside'

    id = Column(Integer, primary_key=True)
    side = Column(String)
    time_in_use = Column(Time)

    #M:M
    loop = relationship('Loop', secondary=bodyside_loop_association, back_populates='bodysides')


class Loop(Base):
    __tablename__ = 'loop'

    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime)
    session_length = Column(Integer) #Seconds
    location = Column(String)
    human_labeled = Column(Boolean)

    #1:M
    videos = relationship('Video', back_populates='loop')

    #M:1
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship('Users', back_populates='loops')

    #M:M
    tools = relationship('Tools', secondary=tools_loop_association, back_populates='loop')
    bodysides = relationship('Bodyside', secondary=bodyside_loop_association, back_populates='loop')


class Video(Base):
    __tablename__= 'video'

    id = Column(Integer, primary_key=True)
    device = Column(String)
    file_path = Column(String)

    #M:1
    loop_id = Column(Integer, ForeignKey('loop.id'))
    loop = relationship('Loop', back_populates='videos')


Base.metadata.create_all(engine)

This notebook will create a new Database with the name: test

<connection object at 0x0000026ED87DB010; dsn: 'user=postgres password=xxx host=localhost port=5432', closed: 0>

FEHLER:  Datenbank »test« existiert bereits

Connected to PostgreSQL, DB: test
2023-10-01 08:55:41,805 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2023-10-01 08:55:41,806 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 08:55:41,809 INFO sqlalchemy.engine.Engine select current_schema()
2023-10-01 08:55:41,810 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 08:55:41,811 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2023-10-01 08:55:41,812 INFO sqlalchemy.engine.Engine [raw sql] {}
2023-10-01 08:55:41,813 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 08:55:41,816 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalo

In [3]:
'''Creating the current know user'''

users_inputs = []

users_inputs.append(Users(name='unknown',surename='unknown'))
users_inputs.append(Users(name='Christina',surename='Greiderer'))
users_inputs.append(Users(name='Christine',surename='Lackinger'))
users_inputs.append(Users(name='Juergen',surename='Zangerl'))
users_inputs.append(Users(name='Lukas',surename='Prenner'))
users_inputs.append(Users(name='Martin',surename='Hofer'))
users_inputs.append(Users(name='Pirmin',surename='Aster'))
users_inputs.append(Users(name='Robert',surename='Goller'))
users_inputs.append(Users(name='Suganthi',surename='Manoharan'))
users_inputs.append(Users(name='Philipp',surename='Egger'))
users_inputs.append(Users(name='MartinPO',surename='Feuerstein'))

session.bulk_save_objects(users_inputs)
session.commit()

2023-10-01 08:55:41,900 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 08:55:41,902 INFO sqlalchemy.engine.Engine INSERT INTO users (name, surename) VALUES (%(name__0)s, %(surename__0)s), (%(name__1)s, %(surename__1)s), (%(name__2)s, %(surename__2)s), (%(name__3)s, %(surename__3)s), (%(name__4)s, %(surename__4)s), (%(name__5)s, %(surename__5)s), (%(name__6)s, %(surename__6)s), (%(name__7)s, %(surename__7)s), (%(name__8)s, %(surename__8)s), (%(name__9)s, %(surename__9)s), (%(name__10)s, %(surename__10)s)
2023-10-01 08:55:41,903 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/1 (unordered)] {'name__0': 'unknown', 'surename__0': 'unknown', 'name__1': 'Christina', 'surename__1': 'Greiderer', 'name__2': 'Christine', 'surename__2': 'Lackinger', 'name__3': 'Juergen', 'surename__3': 'Zangerl', 'name__4': 'Lukas', 'surename__4': 'Prenner', 'name__5': 'Martin', 'surename__5': 'Hofer', 'name__6': 'Pirmin', 'surename__6': 'Aster', 'name__7': 'Robert', 'surename

In [4]:
'''Creating the tool input'''

tools_inputs = []

tools_inputs.append(Tools(tool='trigger'))
tools_inputs.append(Tools(tool='small duoballs'))
tools_inputs.append(Tools(tool='big duoballs'))

session.bulk_save_objects(tools_inputs)
session.commit()

2023-10-01 08:55:41,919 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 08:55:41,920 INFO sqlalchemy.engine.Engine INSERT INTO tools (tool) VALUES (%(tool__0)s), (%(tool__1)s), (%(tool__2)s)
2023-10-01 08:55:41,921 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues) 1/1 (unordered)] {'tool__0': 'trigger', 'tool__1': 'small duoballs', 'tool__2': 'big duoballs'}
2023-10-01 08:55:41,923 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
'''Creating the bodyside input'''

bodyside_inputs = []

bodyside_inputs.append(Bodyside(side='left'))
bodyside_inputs.append(Bodyside(side='middle'))
bodyside_inputs.append(Bodyside(side='right'))

session.bulk_save_objects(bodyside_inputs)
session.commit()

2023-10-01 08:55:41,935 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 08:55:41,936 INFO sqlalchemy.engine.Engine INSERT INTO bodyside (side) VALUES (%(side__0)s), (%(side__1)s), (%(side__2)s)
2023-10-01 08:55:41,937 INFO sqlalchemy.engine.Engine [generated in 0.00006s (insertmanyvalues) 1/1 (unordered)] {'side__0': 'left', 'side__1': 'middle', 'side__2': 'right'}
2023-10-01 08:55:41,939 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
data_doc = pd.read_excel('Session_data.xlsx')

In [7]:
data_doc

,Exercise,Users,BodySide,StartTime,StopTime,CorrectMovement
0,trigger,Martin,left,NaN,NaN,yes
1,trigger,Juergen,left,NaN,NaN,yes
2,trigger,Christina,left,NaN,NaN,yes
3,trigger,Philipp,left,NaN,NaN,yes
4,trigger,Lukas,left,NaN,NaN,yes
...,...,...,...,...,...,...
123,big duoballs,Juergen,middle,2023-09-18 15:52:59.607825,2023-09-18 15:52:59.607825,yes
124,small duoballs,Juergen,middle,2023-09-18 15:54:08.421279,2023-09-18 15:54:08.421279,yes
125,small duoballs,Juergen,middle,2023-09-18 15:55:05.638256,2023-09-18 15:55:05.638256,yes
126,trigger,Juergen,left,2023-09-18 15:56:46.493459,2023-09-18 15:56:46.493459,yes


In [8]:
def get_video_duration(file_path):
    try:
        video_clip = VideoFileClip(file_path)
        duration = video_clip.duration
        video_clip.close()
        return duration
    except Exception as e:
        print(f"Error: {e}")
        return None

In [9]:
def datacollector(source_directory: str, location: str,human_labeled: bool ):
    """Cam Options: leftCam, rightCam, rgbCam"""

    list_of_video_data = []
    counter = 1

    for root, dirs, files in os.walk(source_directory):

        timestamp_str = '-'.join(os.path.basename(root).split('-')[1:])
        try:
            timestamp = datetime.strptime(timestamp_str, "%Y-%m-%d %H_%M_%S.%f")
        except ValueError:
            print(f"Skipping invalid timestamp: {timestamp_str}")
            continue


        duration = get_video_duration(os.path.join(root, files[0]))
        user = session.query(Users.id).filter(Users.name == data_doc['Users'][counter - 1]).first()
        user_id = user[0] if user else None

        tool = session.query(Tools).filter(Tools.tool == data_doc['Exercise'][counter - 1]).first()
        bodyside = session.query(Bodyside).filter(Bodyside.side == data_doc['BodySide'][counter - 1]).first()


        new_loop = Loop(timestamp=timestamp, location=location, session_length=duration, human_labeled=human_labeled, user_id=user_id)
        session.add(new_loop)
        session.commit()

        new_loop.tools.append(tool)
        session.commit()

        new_loop.bodysides.append(bodyside)
        session.commit()

        for filename in files:

            if filename.split(".")[-1] !="mp4":
                filepath = os.path.join(root, filename)
                os.remove(filepath)

            if filename.split(".")[-1] =="mp4":
                    oidjgäpoadjg  gf eanrölnkppojpokä2 
                filepath = filepath.replace("\\", "/")
                print(filepath)
                device = filename.split(".")[0]
                new_video = Video(file_path=filepath,device=device,loop_id=counter)
                session.add(new_video)
                session.commit()


        counter += 1


    print("Done!")


In [10]:
video_data = datacollector('D:/tt_video_data', 'MCI', True)

Skipping invalid timestamp: 
D:\tt_video_data\videos-2023-09-08 19_12_12.778376
2023-10-01 08:55:42,300 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-10-01 08:55:42,302 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id 
FROM users 
WHERE users.name = %(name_1)s 
 LIMIT %(param_1)s
2023-10-01 08:55:42,303 INFO sqlalchemy.engine.Engine [generated in 0.00086s] {'name_1': 'Martin', 'param_1': 1}
2023-10-01 08:55:42,305 INFO sqlalchemy.engine.Engine SELECT tools.id AS tools_id, tools.tool AS tools_tool, tools.time_in_use AS tools_time_in_use 
FROM tools 
WHERE tools.tool = %(tool_1)s 
 LIMIT %(param_1)s
2023-10-01 08:55:42,305 INFO sqlalchemy.engine.Engine [generated in 0.00046s] {'tool_1': 'trigger', 'param_1': 1}
2023-10-01 08:55:42,308 INFO sqlalchemy.engine.Engine SELECT bodyside.id AS bodyside_id, bodyside.side AS bodyside_side, bodyside.time_in_use AS bodyside_time_in_use 
FROM bodyside 
WHERE bodyside.side = %(side_1)s 
 LIMIT %(param_1)s
2023-10-01 08:55:42,308 INFO sq